# Preprocess ODK data to organized tables

# Imports and Set-up

In [2]:
# Standard Imports
import sys
import os
import urllib.request
import pandas as pd
import numpy as np
import re

# Google Cloud Imports
import pandas_gbq

In [3]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import DATA_DIR, GCP_PROJ_ID
from src.biomass_inventory import (
    extract_trees,
    extract_stumps,
    extract_dead_trees_class1,
    extract_dead_trees_c3,
)

In [4]:
# Variables
URL = "https://api.ona.io/api/v1/data/763932.csv"
FILE_RAW = DATA_DIR / "csv" / "biomass_inventory_raw.csv"
CARBON_POOLS_OUTDIR = DATA_DIR / "csv" / "carbon_pools"
NESTS = [2, 3, 4]

# BigQuery Variables
DATASET_ID = "biomass_inventory"

In [ ]:
# Create output directory
CARBON_POOLS_OUTDIR.mkdir(parents=True, exist_ok=True)

# Get Data from ONA

In [5]:
column_types = {
    col: str
    for col in (
        28,
        399,
        400,
        407,
        408,
        415,
        416,
        845,
        846,
        853,
        854,
        861,
        862,
        869,
        870,
        877,
        878,
        885,
        886,
        893,
        894,
        901,
        902,
        909,
        910,
        1179,
        1180,
        1187,
        1188,
        1195,
        1196,
        1203,
        1204,
        1211,
        1212,
        1219,
        1220,
        1286,
        1337,
        1342,
        1347,
        1352,
        1357,
        1362,
        1378,
        1392,
    )
}

In [6]:
if FILE_RAW.exists():
    data = pd.read_csv(FILE_RAW, dtype=column_types)
else:
    urllib.request.urlretrieve(URL, FILE_RAW)
    data = pd.read_csv(FILE_RAW, dtype=column_types)

## Add a unique ID

In [7]:
# Create a new column with "1" for Primary and "2" for Backup
data["plot_info/plot_type_short"] = data["plot_info/plot_type"].apply(
    lambda x: "1" if x == "Primary" else "2"
)

# Extract subplot letters (assuming they are included in the 'plot_info.sub_plot' column)
data["subplot_letter"] = data["plot_info/sub_plot"].str.replace("sub_plot", "")

# Create the unique ID by concatenating the specified columns
data["unique_id"] = (
    data["plot_info/plot_code_nmbr"].astype(str)
    + data["subplot_letter"]
    + data["plot_info/plot_type_short"]
)

# Extract Plot info

In [ ]:
plot_info_cols = [
    "unique_id",
    "plot_info/data_recorder",
    "plot_info/team_no",
    "plot_info/plot_code_nmbr",
    "plot_info/plot_type",
    "plot_info/sub_plot",
    "plot_info/yes_no",
    "plot_shift/sub_plot_shift",
    "plot_GPS/GPS_waypt",
    "plot_GPS/GPS_id",
    "plot_GPS/GPS",
    "plot_GPS/_GPS_latitude",
    "plot_GPS/_GPS_longitude",
    "plot_GPS/_GPS_altitude",
    "plot_GPS/_GPS_precision",
    "plot_GPS/photo",
    "access/access_reason/slope",
    "access/access_reason/danger",
    "access/access_reason/distance",
    "access/access_reason/water",
    "access/access_reason/prohibited",
    "access/access_reason/other",
    "access/manual_reason",
    "lc_data/lc_type",
    "lc_class/lc_class",
    "lc_class/lc_class_other",
    "disturbance/disturbance_yesno",
    "disturbance_data/disturbance_type",
    "disturbance_class/disturbance_class",
    "slope/slope",
    "canopy/avg_height",
    "canopy/can_cov",
]

In [ ]:
plot_info = data[plot_info_cols]

In [ ]:
plot_info.head(2)

# Extract info per carbon pool

# Living Trees

In [ ]:
trees = extract_trees(data, NESTS)

In [ ]:
trees.info(), trees.head(2)

## Export data and upload to BQ

In [ ]:
# Export to CSV
trees.to_csv(CARBON_POOLS_OUTDIR / "trees.csv", index=False)

In [ ]:
# Upload to BQ
pandas_gbq.to_gbq(trees, f"{DATASET_ID}.trees", project_id=GCP_PROJ_ID)

# Tree Stumps

note (delete when addressed): removed `'biomass_per_kg_tree': [biomass_per_kg_tree],`. In the original code there was a placeholder column created, this can be added later in the process when biomass per tree is actually calculated

In [ ]:
stumps = extract_stumps(data, NESTS)

In [ ]:
stumps.info(), stumps.head(2)

## Export data and upload to BQ

In [ ]:
# Export to CSV
stumps.to_csv(CARBON_POOLS_OUTDIR / "stumps.csv", index=False)

In [ ]:
# Upload to BQ
pandas_gbq.to_gbq(stumps, f"{DATASET_ID}.stumps", project_id=GCP_PROJ_ID)

# Dead Trees: Class 1

In [ ]:
dead_trees_c1 = extract_dead_trees_class1(data, NESTS)

In [ ]:
dead_trees_c1.info(), dead_trees_c1.head(2)

## Export data and upload to BQ

In [ ]:
dead_trees_c1.to_csv(CARBON_POOLS_OUTDIR / "dead_trees_class1.csv", index=False)

In [ ]:
# Upload to BQ
pandas_gbq.to_gbq(dead_trees_c1, f"{DATASET_ID}.dead_trees_c1", project_id=GCP_PROJ_ID)

# Dead Trees: Class 2

In [ ]:
dead_trees_c2 = extract_dead_trees_class2(data, NESTS)

In [ ]:
dead_trees_c2.info(), dead_trees_c2.head(2)

## Export data and upload to BQ

In [ ]:
# Export CSV
if len(dead_trees_c2) != 0:
    dead_trees_c2.to_csv(CARBON_POOLS_OUTDIR / "dead_trees_class2.csv", index=False)

In [ ]:
# Upload to BQ
if len(dead_trees_c2) != 0:
    pandas_gbq.to_gbq(
        dead_trees_c2, f"{DATASET_ID}.dead_trees_c2", project_id=GCP_PROJ_ID
    )

# Dead Trees: Class 3

In [8]:
dead_trees_c3 = extract_dead_trees_c3(data, NESTS)

In [9]:
dead_trees_c3.info(), dead_trees_c3.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unique_id     254 non-null    object 
 1   nest          254 non-null    int64  
 2   species_name  201 non-null    float64
 3   family_name   12 non-null     float64
 4   dbh_tall      254 non-null    float64
 5   db_tall       254 non-null    float64
 6   tall_density  254 non-null    float64
 7   slope_t_tall  254 non-null    float64
 8   slope_b_tall  254 non-null    float64
 9   dist_t_tall   254 non-null    float64
 10  class         254 non-null    int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 22.0+ KB


(None,
   unique_id  nest  species_name  family_name  dbh_tall  db_tall  tall_density  \
 0     368A2     2         999.0          NaN      21.9     24.1           1.0   
 1     281A2     2           NaN          NaN      32.6     40.8           1.0   
 
    slope_t_tall  slope_b_tall  dist_t_tall  class  
 0          61.0          56.0         10.0      2  
 1         115.0          20.0          9.3      2  )

## Export data and upload to BQ

In [10]:
# Export CSV
if len(dead_trees_c3) != 0:
    dead_trees_c3.to_csv(CARBON_POOLS_OUTDIR / "dead_trees_class3.csv", index=False)

In [12]:
# Upload to BQ
if len(dead_trees_c3) != 0:
    pandas_gbq.to_gbq(
        dead_trees_c3, f"{DATASET_ID}.dead_trees_c3", project_id=GCP_PROJ_ID
    )

100%|██████████| 1/1 [00:00<00:00, 10205.12it/s]


# Lying Deadwood: Hollow

In [48]:
ldw_hollow = extract_ldw_with_hollow(data)

In [49]:
ldw_hollow.info(), ldw_hollow.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   15 non-null     object 
 1   repetition  15 non-null     int64  
 2   type        15 non-null     object 
 3   class       15 non-null     object 
 4   hollow_d1   15 non-null     float64
 5   hollow_d2   15 non-null     float64
 6   diameter    15 non-null     float64
 7   density     15 non-null     float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.1+ KB


(None,
   unique_id  repetition type class  hollow_d1  hollow_d2  diameter  density
 0     249D2           1  tr2   MDF       12.0       10.0      16.7      3.0
 1     290A2           2  tr2   MCB       68.0       28.0      62.0      1.0)

## Export data and upload to BQ

In [51]:
# Export CSV
if len(ldw_hollow) != 0:
    ldw_hollow.to_csv(CARBON_POOLS_OUTDIR / "lying_deadwood_hollow.csv", index=False)

In [52]:
# Upload to BQ
if len(ldw_hollow) != 0:
    pandas_gbq.to_gbq(
        ldw_hollow, f"{DATASET_ID}.lying_deadwood_hollow", project_id=GCP_PROJ_ID
    )

100%|██████████| 1/1 [00:00<00:00, 14665.40it/s]
